# 01. データ前処理

このノートブックでは、実データ（df_node.pkl、df_element.pkl）からRIMD特徴量の計算と標準化を行います。

## 処理内容
1. 実データ（df_node.pkl、df_element.pkl）の読み込み
2. ケース別データの抽出と処理
3. RIMD特徴量の計算
4. データ分割（train/val/test）
5. 標準化器のフィット
6. 処理済みデータの保存

## データ構造
- **df_node.pkl**: ノード単位のxyz座標値
  - 1step法: step_blk_coord_x/y, step_prod_coord_x/y
  - 逐次解析: nv_blk_coord_x/y, nv_prod_coord_x/y
  - NodeIDがケースごとに1~1071
- **df_element.pkl**: 要素単位の構成情報（ElementID n1 n2 n3 n4）
- 両方ともcaseカラムでケースNo（No001, No002, ...）を識別

In [ ]:
# 必要なライブラリのインポート
import sys
from pathlib import Path

# プロジェクトルートをパスに追加
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from src.data.preprocessing import RIMDPreprocessor
from src.utils.experiment_manager import ExperimentManager
from config.experiment_configs import get_baseline_config
import logging

# ログレベル設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# 設定読み込み
config = get_baseline_config()
print(f"実験ID: {config.exp_id}")
print(f"説明: {config.description}")

# 実験マネージャー作成
exp_manager = ExperimentManager(config)
print(f"実験ディレクトリ: {exp_manager.exp_dir}")

In [ ]:
# データ前処理器を初期化
preprocessor = RIMDPreprocessor(config.data)
print("RIMDPreprocessor initialized")

In [ ]:
# 全ケースの前処理を実行
print("開始: 全ケースの前処理")
try:
    preprocessor.process_all_cases()
    print("✅ 全ケースの前処理完了")
except Exception as e:
    print(f"❌ 前処理エラー: {e}")
    raise

In [ ]:
# データ分割を作成
print("開始: データ分割作成")
try:
    preprocessor.create_splits()
    splits = preprocessor.load_splits()
    
    print("✅ データ分割完了")
    print(f"Train: {len(splits['train'])} cases")
    print(f"Val: {len(splits['val'])} cases")
    print(f"Test: {len(splits['test'])} cases")
    
except Exception as e:
    print(f"❌ データ分割エラー: {e}")
    raise

In [ ]:
# 標準化器をフィットして保存
print("開始: 標準化器のフィット")
try:
    scalers = preprocessor.fit_scalers()
    exp_manager.save_scalers(scalers)
    
    print("✅ 標準化器のフィット完了")
    print(f"代表寸法: {scalers['representative_scale']:.2f}")
    print("スケーラが保存されました")
    
except Exception as e:
    print(f"❌ 標準化器フィットエラー: {e}")
    raise

In [ ]:
# 処理結果の確認
from src.data.dataset import RIMDDataModule

print("処理結果確認...")
try:
    # データモジュール作成
    datamodule = RIMDDataModule(config.data, scalers)
    
    # サンプルバッチ取得
    sample_batch = datamodule.get_sample_batch("train")
    
    print("✅ データモジュール動作確認完了")
    print(f"エッジ特徴量次元: {sample_batch['edge_features'].shape}")
    print(f"ノード特徴量次元: {sample_batch['node_features'].shape}")
    print(f"ターゲット次元: {sample_batch['target'].shape}")
    
except Exception as e:
    print(f"❌ データモジュール確認エラー: {e}")
    raise

## 前処理完了

前処理が正常に完了しました。次に `02_training.ipynb` で学習を開始できます。

### 処理された内容
- ✅ RIMD特徴量計算
- ✅ データ分割（train/val/test）
- ✅ 標準化器フィット
- ✅ 処理済みデータ保存

### 次のステップ
1. `02_training.ipynb` でモデル学習
2. `03_evaluation.ipynb` でモデル評価
3. `04_analysis.ipynb` で結果分析